In [2]:
import os
import sys
sys.path.append('../')
import glob2 as glob
import json
import requests
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

In [3]:
# Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import ShuffleSplit, KFold

In [4]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [5]:
test_features = pd.read_csv(('data2/dpc_test_features.csv'), 
                                   index_col='match_id')

X_test = test_features.values
y_test_pred = model.predict_proba(X_test)[:, 1] ## NOTE CHANGE 'MODEL',predict with the name

df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=df_test_features.index)


NameError: name 'model' is not defined

## Random Forest 

Try doing logistic regression with just hot encoded heroes and see what results are like

In [49]:
SEED = 17

In [50]:
# Import Training and Test Data  
train_features = pd.read_csv('data2/dpc_train_features.csv', index_col='match_id')
train_targets = pd.read_csv('data2/dpc_train_target.csv', index_col='match_id')
test_features = pd.read_csv('data2/dpc_test_features.csv', index_col='match_id')
test_targets = pd.read_csv('data2/dpc_test_target.csv', index_col='match_id')

In [51]:
train_features.shape, train_targets.shape, test_features.shape, test_targets.shape

((1045, 66), (1045, 2), (117, 66), (117, 2))

In [9]:
df_full_features = pd.concat([train_features, test_features])

# Index to split the training and test data sets
idx_split = train_features.shape[0]

heroes_df = df_full_features[[f'{i}hero_id' for i in range(1, 23)]]

In [10]:
# Quick check that hero ids are unique (i.e., no double picking)
# https://www.kaggle.com/kuzand/dota-2-winner-prediction-multilayer-nn-pytorch
np.all(df_full_features[[f'{i}hero_id' for i in range(1, 23)]].nunique(axis=1) == 22)

True

Hot encoding by getting dummies for each hero_id feature. 

In [11]:
for i in range(1,23):
        df_full_features = pd.get_dummies(df_full_features, columns = [f'{i}hero_id'])

### Scaling 

Trying with standard scaling

In [12]:
# Standard Scaling

df_full_features_scaled = df_full_features.copy()
df_full_features_scaled[df_full_features.columns.tolist()] = StandardScaler().fit_transform(df_full_features_scaled[df_full_features.columns.tolist()])  # alternatively use StandardScaler

In [43]:
df_full_features_scaled.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4865407245,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [13]:
df_full_features_scaled.shape

(1162, 2088)

## Set targets, features, and verification set 

Standard Scale

In [34]:
# Encoded/Scaled train/test features

X_train1 = df_full_features_scaled[:idx_split]
X_test1 = df_full_features_scaled[idx_split:]

X_train = df_full_features_scaled[:idx_split]
X_test = df_full_features_scaled[idx_split:]

y_train = train_targets.radiant_win
y_test = test_targets.radiant_win

In [47]:
X_train.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4865407245,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Train Validation Split

Train Validation Split

For now, let's just try this out with the hero_ohe feature set

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [16]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((940, 2088), (105, 2088), (940,), (105,))

## Setup/Pre-process Summary

Let's try random forest classifier with hot encoded data (i.e., dummy variables on hero id's), StandardScaling

## RFC Attempt 1. 

For the first pass, let's try the Random Forest model with the entire training set (i.e., will not split training data)

Fitting the random forest model to the training set. Checking performance as we adjust the number of trees. 

In [21]:
# Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import ShuffleSplit, KFold

In [3]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [25]:
from sklearn.ensemble import RandomForestClassifier


In [5]:
test_features = pd.read_csv(('data2/dpc_test_features.csv'), 
                                   index_col='match_id')

X_test = test_features.values
y_test_pred = model.predict_proba(X_test)[:, 1] ## NOTE CHANGE 'MODEL',predict with the name

df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=df_test_features.index)


NameError: name 'model' is not defined

## Random Forest 

Try doing logistic regression with just hot encoded heroes and see what results are like

In [26]:
SEED = 17

In [27]:
# Import Training and Test Data  
train_features = pd.read_csv('data2/dpc_train_features.csv', index_col='match_id')
train_targets = pd.read_csv('data2/dpc_train_target.csv', index_col='match_id')
test_features = pd.read_csv('data2/dpc_test_features.csv', index_col='match_id')
test_targets = pd.read_csv('data2/dpc_test_target.csv', index_col='match_id')

In [28]:
train_features.shape, train_targets.shape, test_features.shape, test_targets.shape

((1045, 66), (1045, 2), (117, 66), (117, 2))

In [29]:
df_full_features = pd.concat([train_features, test_features])

# Index to split the training and test data sets
idx_split = train_features.shape[0]

heroes_df = df_full_features[[f'{i}hero_id' for i in range(1, 23)]]

In [30]:
# Quick check that hero ids are unique (i.e., no double picking)
# https://www.kaggle.com/kuzand/dota-2-winner-prediction-multilayer-nn-pytorch
np.all(df_full_features[[f'{i}hero_id' for i in range(1, 23)]].nunique(axis=1) == 22)

True

Hot encoding by getting dummies for each hero_id feature. 

In [31]:
for i in range(1,23):
        df_full_features = pd.get_dummies(df_full_features, columns = [f'{i}hero_id'])

### Scaling 

Trying it out with minmax

In [42]:
## MINMAX SCALING 

df_full_features_scaled = df_full_features.copy()
df_full_features_scaled[df_full_features.columns.tolist()] = MinMaxScaler().fit_transform(df_full_features_scaled[df_full_features.columns.tolist()])  # alternatively use StandardScaler

In [43]:
df_full_features_scaled.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4865407245,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [44]:
df_full_features_scaled.shape

(1162, 2088)

## Set targets, features, and verification set 

Minmax scaled, 

In [45]:
# Encoded train/test features

X_train = df_full_features_scaled[:idx_split]
X_test = df_full_features_scaled[idx_split:]

y_train = train_targets.radiant_win
y_test = test_targets.radiant_win

In [46]:
#X_train.shape, X_train1.shape, y_train.shape, X_test.shape, X_test1.shape, y_test.shape

In [47]:
X_train.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4865407245,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Train Validation Split

Train Validation Split

For now, let's just try this out with the hero_ohe feature set

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [49]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((940, 2088), (105, 2088), (940,), (105,))

## Setup/Pre-process Summary

Let's try random forest classifier with hot encoded data (i.e., dummy variables on hero id's), Minmax scaling

## RFC Attempt 1. 

For the first pass, let's try the Random Forest model with the entire training set (i.e., will not split training data)

Fitting the random forest model to the training set. Checking performance as we adjust the number of trees. 

In [17]:
# Try this once with 10 estimators 

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10, n_jobs=4, max_depth=None, random_state=SEED)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
                       oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [23]:
# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred = model.predict_proba(X_valid)[:, 1]

In [24]:
y_pred

array([0.3, 0.4, 0.8, 0.6, 0.4, 0.4, 0.4, 0.3, 0.4, 0.7, 0.4, 0.4, 0.6,
       0.8, 0.5, 0.6, 0.9, 0.4, 0.7, 0.5, 0.2, 0.4, 0.4, 0.3, 0.7, 0.7,
       0.4, 0.7, 0.4, 0.3, 0.5, 0.7, 0.4, 0.5, 0.8, 0.3, 0.3, 0.3, 0.9,
       0.2, 0.7, 0.4, 0.7, 0.6, 0.5, 0.5, 0.4, 0.9, 0.6, 0.5, 0.9, 0.6,
       0.7, 0.5, 0.7, 0.5, 0.9, 0.3, 0.3, 0.7, 0.7, 0.7, 0.9, 0.8, 0.5,
       0.5, 0.8, 0.7, 0.5, 0.3, 0.3, 0.7, 0.5, 0.2, 0.7, 0.8, 0.1, 0.3,
       0.8, 0.7, 0.4, 0.6, 0.7, 0.5, 0.1, 0.7, 0.6, 0.8, 0.4, 0.5, 0.3,
       0.3, 0.4, 0.4, 0.3, 0.6, 0.8, 0.4, 0.9, 0.6, 0.6, 0.5, 0.4, 0.5,
       0.7])

In [25]:
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.3588798820928518


In [26]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.41904761904761906


In [31]:
from xgboost import XGBClassifier

XGB_model = XGBClassifier(n_jobs=3, random_state=SEED)
RF_model = RandomForestClassifier(n_jobs=3, random_state=SEED)

RF_model.fit(X_train, y_train)
XGB_model.fit(X_train, y_train)

print(f"Random Forest score: {RF_model.score(X_valid, y_valid)}")
print(f"XG Boost score: {XGB_model.score(X_valid, y_valid)}")

Random Forest score: 0.38095238095238093
XG Boost score: 0.41904761904761906


## Cross Validation

Try with a 5 fold shuffle split. On ENTIRE training set 

In [35]:
# Encoded/Scaled train/test features

X_train1 = df_full_features_scaled[:idx_split]
X_test1 = df_full_features_scaled[idx_split:]

X_train = df_full_features_scaled[:idx_split]
X_test = df_full_features_scaled[idx_split:]

y_train = train_targets.radiant_win
y_test = test_targets.radiant_win

In [36]:
from sklearn.model_selection import ShuffleSplit, KFold
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=SEED)

In [38]:
%%time
model_rf1 = RandomForestClassifier(n_estimators=100, n_jobs=4,
                                   max_depth=None, random_state=17)

# calcuate ROC-AUC for each split
cv_scores_rf1 = cross_val_score(model_rf1, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)

Wall time: 4.48 s


In [40]:
%%time

model_rf2 = RandomForestClassifier(n_estimators=100, n_jobs=4,
                                   min_samples_leaf=3, random_state=17)

cv_scores_rf2 = cross_val_score(model_rf2, X_train, y_train, cv=cv, 
                                scoring='roc_auc', n_jobs=-1)

Wall time: 2.94 s


In [42]:
cv_scores_rf1, cv_scores_rf2

(array([0.38025055, 0.45435917, 0.43645606, 0.4247076 , 0.44448476]),
 array([0.39498895, 0.47280179, 0.41394336, 0.40643275, 0.41219158]))

In [43]:
print('Model 1 mean score:', cv_scores_rf1.mean())
print('Model 2 mean score:', cv_scores_rf2.mean())

Model 1 mean score: 0.4280516289766436
Model 2 mean score: 0.42007168404823253


In [44]:
cv_scores_rf2 > cv_scores_rf1

array([ True,  True, False, False, False])

In [23]:
# https://www.kaggle.com/vladislavkisin/dota-2-new-approaches-model-comparison-json
#Count CV scoring and houldout scoring: 
holdout_score_1 = roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1])
print('Holdout score: ', holdout_score_1)

Holdout score:  0.47310243183493006


In [ ]:
cv_score_1 = cross_val_score(model, full_df_mod[:idx_split], y_train, cv=cv, scoring = 'roc_auc') 

In [ ]:
# https://www.kaggle.com/vladislavkisin/dota-2-new-approaches-model-comparison-json
#New results.
print('CV scores: ', cv_score_1)
print('CV mean: ', cv_score_1.mean())
print('CV std:', cv_score_1.std())
print('Holdout score: ', holdout_score_1)
print('Better results on CV: ', cv_score_1>cv_score)